## Loading Libraries and Reading Data

In [1]:
import json
import pandas as pd

In [83]:
data = open("../data/PGCR_graph_dataset0.json")
data = json.load(data)

In [ ]:
data_copy = data

In [4]:
data[1]

[{'ErrorCode': 1,
  'ErrorStatus': 'Success',
  'Message': 'Ok',
  'MessageData': {},
  'Response': {'activityDetails': {'directorActivityHash': 3616746132,
    'instanceId': '1467313524',
    'isPrivate': False,
    'mode': 43,
    'modes': [5, 19, 43],
    'referenceId': 374818561},
   'entries': [{'characterId': '2305843009319880932',
     'extended': {'values': {'allMedalsEarned': {'basic': {'displayValue': '17',
         'value': 17.0}},
       'medalAbilityGunslingerQuick': {'basic': {'displayValue': '1',
         'value': 1.0}},
       'medalAvenger': {'basic': {'displayValue': '1', 'value': 1.0}},
       'medalControlAdvantageHold': {'basic': {'displayValue': '2',
         'value': 2.0}},
       'medalControlAdvantageStreak': {'basic': {'displayValue': '1',
         'value': 1.0}},
       'medalControlPowerPlayWipe': {'basic': {'displayValue': '1',
         'value': 1.0}},
       'medalMatchBlowout': {'basic': {'displayValue': '1', 'value': 1.0}},
       'medalMulti2x': {'basic

## Exploring the structure of data

In [84]:
# Exploring the structure of data
len(data) # 1003, is a list
len(data[0]) # 25, is a list
data[0][0]["Response"]["entries"][0]["values"]["fireteamId"]["basic"]["value"] # fire team ID
data[0][0]["Response"]["entries"][0]["player"]["destinyUserInfo"]["membershipId"] # destiny user ID

'4611686018445743058'

In [57]:
# game_set 386 has some issues
for i, game_set in enumerate(data):
    print(i)
    if i == 386 : continue
    for game_info in game_set:
        for player_info in game_info["Response"]["entries"]:
            print(player_info["player"]["destinyUserInfo"]["membershipId"])

In [61]:
# removing the 386th element in data
elem = data.pop(386)

'4611686018445743058'

## Extracting `playerIDs` and `fireTeamIDs`

In [59]:
playerIDs = [player_info["player"]["destinyUserInfo"]["membershipId"] for game_set in data for game_info in game_set for player_info in game_info["Response"]["entries"]]
len(playerIDs)

121645

In [60]:
fireTeamIDs = [player_info["values"]["fireteamId"]["basic"]["value"] for game_set in data for game_info in game_set for player_info in game_info["Response"]["entries"]]
len(fireTeamIDs)

121645

## Creating the Data Frame

In [63]:
T3 = pd.DataFrame({"fireTeamID" : fireTeamIDs, "playerID" : playerIDs})
T3.head()

,fireTeamID,playerID
0,-3.685548e+17,4611686018445743058
1,-2.245961e+18,4611686018445765559
2,-5.620820e+18,4611686018436434230
3,1.855335e+17,4611686018446767615
4,3.666009e+18,4611686018455684505


## What is wrong with elem 386?

Using the for loop to investigate the problem, we found that `elem[4]` had some issues.

In [76]:
len(elem)

for i, game_info in enumerate(elem):
    print(i)
    for player_info in game_info["Response"]["entries"]:
        print(player_info["values"]["fireteamId"]["basic"]["value"])

0
1.600697195277455e+18
-6.348326376293105e+18
-6.494090189096198e+18
1.1028136067465902e+17
6.345618395897744e+18
7.292249995381676e+18
6.345618395897744e+18
6.345618395897744e+18
1
-6.784968767515582e+18
-6.494090189096198e+18
-6.54366773945844e+18
8.045228988200403e+18
-5.741375587544479e+18
2.5781988903164544e+18
-7.396708596824929e+18
4.173493896829916e+18
2
7.475683580473355e+18
-1.7666294348954877e+17
2.4083941538884538e+17
-6.899599156010211e+18
-7.170965067668722e+18
-6.494090189096198e+18
-2.70978892716923e+18
2.4083941538884538e+17
3
7.35842355099692e+18
4.819254210965579e+18
6.919517908570447e+18
1.8524873921183473e+18
-1.6663537611408812e+18
-6.494090189096198e+18
2.0248044766801326e+18
-2.075682527226918e+18
-4.410787300159699e+18
4


KeyError: 'Response'

`elem[4]` has non unit `ErrorCode`.

In [82]:
elem[4]

{'ErrorCode': 1618,
 'ErrorStatus': 'DestinyUnexpectedError',
 'Message': "An unexpected error has occurred on Bungie's servers while trying to grab Destiny information.",
 'MessageData': {},
 'ThrottleSeconds': 0}

Clearly, we also need to consider the value of the `ErrorCode` before extracting the information to avoid such errors.

## Modifying the extraction code

In [ ]:
data = data_copy

In [86]:
playerIDs = [player_info["player"]["destinyUserInfo"]["membershipId"] for game_set in data for game_info in game_set if game_info["ErrorCode"] == 1 for player_info in game_info["Response"]["entries"]]
len(playerIDs)

121744

In [87]:
fireTeamIDs = [player_info["values"]["fireteamId"]["basic"]["value"] for game_set in data for game_info in game_set if game_info["ErrorCode"] == 1 for player_info in game_info["Response"]["entries"]]
len(fireTeamIDs)

121744

In [88]:
T3 = pd.DataFrame({"fireTeamID" : fireTeamIDs, "playerID" : playerIDs})
T3.head()

,fireTeamID,playerID
0,-3.685548e+17,4611686018445743058
1,-2.245961e+18,4611686018445765559
2,-5.620820e+18,4611686018436434230
3,1.855335e+17,4611686018446767615
4,3.666009e+18,4611686018455684505
